In [1]:
from sympy import *
from sympy.physics.quantum import TensorProduct
from sympy.physics.quantum import Commutator
from sympy.physics.quantum import Operator
from itertools import permutations
from itertools import combinations

In [2]:
wc_0,wc_1,wq1_0,wq1_1,wq2_0,wq2_1,g12_00,g1c_00,g2c_00,t = symbols('w^c_0,w^c_1,w^1_0,w^1_1,w^2_0,w^2_1,g^(12)_00,g^(1c)_00,g^(2c)_00,t')
wc_2,wq1_2,wq2_2,g12_01,g12_10,g12_11,g1c_01,g1c_10,g1c_11,g2c_01,g2c_10,g2c_11 = symbols('w^c_2,w^1_2,w^2_2,g^(12)_01,g^(12)_10,g^(12)_11,g^(1c)_01,g^(1c)_10,g^(1c)_11,g^(2c)_01,g^(2c)_10,g^(2c)_11')
d1c_00,d1c_01,d1c_10,d1c_11, d2c_00,d2c_01,d2c_10,d2c_11= symbols('D^(1c)_00,D^(1c)_01,D^(1c)_10,D^(1c)_11,D^(2c)_00,D^(2c)_01,D^(2c)_10,D^(2c)_11')
d12_00,d12_01,d12_10,d12_11 = symbols('D^(12)_00,D^(12)_01,D^(12)_10,D^(12)_11')

sigmap0 = Matrix([[0,0,0],[1,0,0],[0,0,0]])
sigmap1 = Matrix([[0,0,0],[0,0,0],[0,1,0]])
sigmam0 = Matrix([[0,1,0],[0,0,0],[0,0,0]])
sigmam1 = Matrix([[0,0,0],[0,0,1],[0,0,0]])
sigmap02 = Matrix([[0,0,0],[0,0,0],[1,0,0]])
sigmam02 = Matrix([[0,0,1],[0,0,0],[0,0,0]])


H0 = TensorProduct(diag(wc_0,wc_1,wc_2),eye(3),eye(3)) + TensorProduct(eye(3),diag(wq1_0,wq1_1,wq1_2),eye(3))+ TensorProduct(eye(3),eye(3),diag(wq2_0,wq2_1,wq2_2))
H1_00 = g12_00*(TensorProduct(eye(3),sigmap0,sigmam0)+TensorProduct(eye(3),sigmam0,sigmap0))
H1_01 = g12_01*(TensorProduct(eye(3),sigmap0,sigmam1)+TensorProduct(eye(3),sigmam0,sigmap1))
H1_10 = g12_10*(TensorProduct(eye(3),sigmap1,sigmam0)+TensorProduct(eye(3),sigmam1,sigmap0))
H1_11 = g12_11*(TensorProduct(eye(3),sigmap1,sigmam1)+TensorProduct(eye(3),sigmam1,sigmap1))
# H1 = H1_00 + H1_01 + H1_10 + H1_11
H1 = H1_00

H2_00 = g1c_00*(TensorProduct(sigmam0,sigmap0,eye(3))+TensorProduct(sigmap0,sigmam0,eye(3)))+g2c_00*(TensorProduct(sigmam0,eye(3),sigmap0)+TensorProduct(sigmap0,eye(3),sigmam0))
H2_01 = g1c_01*(TensorProduct(sigmam0,sigmap1,eye(3))+TensorProduct(sigmap0,sigmam1,eye(3)))+g2c_01*(TensorProduct(sigmam0,eye(3),sigmap1)+TensorProduct(sigmap0,eye(3),sigmam1))
H2_10 = g1c_10*(TensorProduct(sigmam1,sigmap0,eye(3))+TensorProduct(sigmap1,sigmam0,eye(3)))+g2c_10*(TensorProduct(sigmam1,eye(3),sigmap0)+TensorProduct(sigmap1,eye(3),sigmam0))
H2_11 = g1c_11*(TensorProduct(sigmam1,sigmap1,eye(3))+TensorProduct(sigmap1,sigmam1,eye(3)))+g2c_11*(TensorProduct(sigmam1,eye(3),sigmap1)+TensorProduct(sigmap1,eye(3),sigmam1))
H2_qq_01 = g12_01*(TensorProduct(eye(3),sigmap0,sigmam1)+TensorProduct(eye(3),sigmam0,sigmap1))
H2_qq_10 = g12_10*(TensorProduct(eye(3),sigmap1,sigmam0)+TensorProduct(eye(3),sigmam1,sigmap0))
H2_qq_11 = g12_11*(TensorProduct(eye(3),sigmap1,sigmam1)+TensorProduct(eye(3),sigmam1,sigmap1))
H2_qq_00 = g12_00*(TensorProduct(eye(3),sigmap0,sigmam0)+TensorProduct(eye(3),sigmam0,sigmap0))  #
# H2 = H2_00 + H2_01+ H2_10+ H2_11
H2 = H2_00 + H2_01 + H2_10 + H2_11+ H1_01 + H1_10 + H1_11

In [3]:
sub2q_eigen = []
subrest_eigen = []

for c in [[[1],[0],[0]],[[0],[1],[0]],[[0],[0],[1]]]:
    for q1 in [[[1],[0],[0]],[[0],[1],[0]],[[0],[0],[1]]]:
        for q2 in [[[1],[0],[0]],[[0],[1],[0]],[[0],[0],[1]]]:
            if (c == [[1],[0],[0]]) and (q1 != [[0],[0],[1]]) and (q2 != [[0],[0],[1]]):
                sub2q_eigen.append(TensorProduct(Matrix(c),Matrix(q1),Matrix(q2)))
            else:
                subrest_eigen.append(TensorProduct(Matrix(c),Matrix(q1),Matrix(q2)))

P2q = diag([0]*27,unpack=True)
Prest = diag([0]*27,unpack=True)
for k in [i*i.transpose() for i in sub2q_eigen]:
    P2q += k
for k in [i*i.transpose() for i in subrest_eigen]:
    Prest += k


In [4]:
S1_try = diag([0]*27,unpack=True)
for subpairs in permutations([sub2q_eigen,subrest_eigen],2):
    for i in subpairs[0]:
        for j in subpairs[1]:
            S1_try += -(i.transpose()*H2*j)[0]/(i.transpose()*H0*i-j.transpose()*H0*j)[0]*i*j.transpose()

In [5]:
S2_try = diag([0]*27,unpack=True)
for subpairs in permutations([sub2q_eigen,subrest_eigen],2):
    for i in subpairs[0]:
        for j in subpairs[1]:
            S2_try += -((i.transpose()*(H1*S1_try-S1_try*H1)*j)[0]+0.5*(i.transpose()*(H2*S1_try-S1_try*H2)*j)[0])/(i.transpose()*H0*i-j.transpose()*H0*j)[0]*i*j.transpose()

In [6]:
res_Scut1 = H0 + H1 + (H2*S1_try-S1_try*H2)/2
res_Scut2 = H0 + H1 + (H2*S1_try-S1_try*H2)/2 + (H2*S2_try-S2_try*H2)/2


### 等效耦合强度 @ Coupler = $|0\rangle$

In [7]:
res_Scut1[3,1]

g^(12)_00 - g^(1c)_00*g^(2c)_00/(2*(w^2_0 - w^2_1 - w^c_0 + w^c_1)) + g^(1c)_00*g^(2c)_00/(2*(-w^1_0 + w^1_1 + w^c_0 - w^c_1))

In [8]:
res_Scut2[3,1]

-g^(12)_00*g^(1c)_00**2/(2*(w^1_0 - w^1_1 - w^c_0 + w^c_1)*(w^2_0 - w^2_1 - w^c_0 + w^c_1)) - g^(12)_00*g^(2c)_00**2/(2*(-w^1_0 + w^1_1 + w^c_0 - w^c_1)*(-w^2_0 + w^2_1 + w^c_0 - w^c_1)) + g^(12)_00 - g^(1c)_00*g^(2c)_00/(2*(w^2_0 - w^2_1 - w^c_0 + w^c_1)) + g^(1c)_00*g^(2c)_00/(2*(-w^1_0 + w^1_1 + w^c_0 - w^c_1))

### 等效耦合强度 @ Coupler = $|1\rangle$

In [9]:
res_Scut1[10,12]

g^(12)_00 - g^(1c)_00*g^(2c)_00/(2*(-w^2_0 + w^2_1 + w^c_0 - w^c_1)) + g^(1c)_00*g^(2c)_00/(2*(w^1_0 - w^1_1 - w^c_0 + w^c_1))

In [ ]:
simplify(res_Scut2[10,12])

### ZZ强度 @ Coupler = $|0\rangle$

In [11]:
(res_Scut1[4,4] - res_Scut1[3,3]) - (res_Scut1[1,1] - res_Scut1[0,0]) 

-g^(12)_01**2/(2*(w^1_0 - w^1_1 - w^2_1 + w^2_2)) + g^(12)_01**2/(2*(-w^1_0 + w^1_1 + w^2_1 - w^2_2)) + g^(12)_10**2/(2*(w^1_1 - w^1_2 - w^2_0 + w^2_1)) - g^(12)_10**2/(2*(-w^1_1 + w^1_2 + w^2_0 - w^2_1))

In [12]:
(res_Scut2[4,4] - res_Scut2[3,3]) - (res_Scut2[1,1] - res_Scut2[0,0])

g^(12)_00*g^(1c)_00*g^(2c)_00/((w^1_0 - w^1_1 - w^c_0 + w^c_1)*(w^2_0 - w^2_1 - w^c_0 + w^c_1)) + g^(12)_00*g^(1c)_00*g^(2c)_00/((-w^1_0 + w^1_1 + w^c_0 - w^c_1)*(-w^2_0 + w^2_1 + w^c_0 - w^c_1)) - g^(12)_01**2/(2*(w^1_0 - w^1_1 - w^2_1 + w^2_2)) + g^(12)_01**2/(2*(-w^1_0 + w^1_1 + w^2_1 - w^2_2)) + 0.25*g^(12)_01*g^(1c)_00*g^(2c)_01/((w^1_0 - w^1_1 - w^2_1 + w^2_2)*(w^1_0 - w^1_1 - w^c_0 + w^c_1)) + 0.25*g^(12)_01*g^(1c)_00*g^(2c)_01/((-w^1_0 + w^1_1 + w^2_1 - w^2_2)*(-w^1_0 + w^1_1 + w^c_0 - w^c_1)) + g^(12)_10**2/(2*(w^1_1 - w^1_2 - w^2_0 + w^2_1)) - g^(12)_10**2/(2*(-w^1_1 + w^1_2 + w^2_0 - w^2_1)) + 0.25*g^(12)_10*g^(1c)_01*g^(2c)_00/((w^1_1 - w^1_2 - w^2_0 + w^2_1)*(-w^2_0 + w^2_1 + w^c_0 - w^c_1)) + 0.25*g^(12)_10*g^(1c)_01*g^(2c)_00/((-w^1_1 + w^1_2 + w^2_0 - w^2_1)*(w^2_0 - w^2_1 - w^c_0 + w^c_1)) - g^(1c)_00*(-g^(12)_00*g^(2c)_00/(-w^2_0 + w^2_1 + w^c_0 - w^c_1) - 0.5*g^(12)_01*g^(2c)_01/(-w^1_0 + w^1_1 + w^2_1 - w^2_2))/(2*(-w^1_0 + w^1_1 + w^c_0 - w^c_1)) + g^(1c)_00*(g^(12

In [ ]:
simplify((res_Scut2[4,4] - res_Scut2[3,3]) - (res_Scut2[1,1] - res_Scut2[0,0]))
